In [1]:
import os
from typing import List, Dict, Optional
from pydantic import BaseModel, ConfigDict
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain.prompts import PromptTemplate
import json

class MenuItem(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    urun: str
    Fiyat: float
    ingredients: str
    category: str

class Order(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    items: List[Dict[str, any]] = []
    total: float = 0.0

class RestaurantRAG:
    def __init__(self, menu_path: str):
        """Initialize the RestaurantRAG system"""
        self.menu_path = menu_path
        self.menu_items = self.load_menu()
        self.vector_store = self.create_vector_store()
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer",
            input_key="human_input"
        )
        self.current_order = Order()
        self.setup_rag_chain()

    def load_menu(self) -> List[MenuItem]:
        """Load menu items from CSV file"""
        df = pd.read_csv(self.menu_path, delimiter=';')
        menu_items = []
        for _, row in df.iterrows():
            menu_items.append(
                MenuItem(
                    urun=row['urun'],
                    Fiyat=float(row['Fiyat']),
                    ingredients=row['ingredients'],
                    category=row['category']
                )
            )
        return menu_items

    def create_vector_store(self):
        """Create FAISS vector store from menu items"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        
        menu_texts = []
        for item in self.menu_items:
            text = f"Category: {item.category}\nItem: {item.urun}\n"
            text += f"Price: {item.Fiyat} TL\nIngredients: {item.ingredients}\n"
            menu_texts.append(text)

        texts = text_splitter.create_documents(menu_texts)
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_documents(texts, embeddings)
        return vector_store

    def setup_rag_chain(self):
        """Setup the RAG chain with custom prompt"""
        template = """You are a helpful and friendly Turkish restaurant waiter. Use the following information to help the customer.

        Menu Context:
        {context}

        Current Order Status:
        {current_order}

        Chat History:
        {chat_history}

        Customer: {question}
        {human_input}

        Instructions:
        1. Help customers understand the menu and make recommendations
        2. Process orders naturally through conversation
        3. Answer questions about ingredients and prices
        4. Keep track of the current order
        5. Be friendly and professional
        6. Respond in Turkish language
        7. When listing menu items, include their prices and ingredients
        8. Make suggestions based on customer preferences
        9. Handle special requests and modifications professionally

        If the customer wants to add or remove items, respond with a JSON structure:
        {{"add_items": [{{"urun": "item_name", "quantity": number}}], "remove_items": [...], "response": "Your friendly response"}}

        For regular queries about menu or general questions, just respond normally in Turkish.

        Assistant:"""

        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question", "chat_history", "current_order","human_input"]
        )

        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=ChatOpenAI(temperature=0.7, model="gpt-4"),
            retriever=self.vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 5}
            ),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": PROMPT},
            return_source_documents=True,
            chain_type="stuff",
            verbose=True
        )

    def add_to_order(self, item_name: str, quantity: int = 1) -> str:
        """Add item to current order"""
        for menu_item in self.menu_items:
            if menu_item.urun.lower() == item_name.lower():
                order_item = {
                    "urun": menu_item.urun,
                    "quantity": quantity,
                    "Fiyat": menu_item.Fiyat,
                    "total": menu_item.Fiyat * quantity
                }
                self.current_order.items.append(order_item)
                self.current_order.total += order_item["total"]
                return f"Added {quantity}x {menu_item.urun} to your order."
        return f"Sorry, couldn't find {item_name} in our menu."

    def remove_from_order(self, item_name: str, quantity: int = None) -> str:
        """Remove item from current order"""
        for idx, item in enumerate(self.current_order.items):
            if item["urun"].lower() == item_name.lower():
                if quantity is None or quantity >= item["quantity"]:
                    self.current_order.total -= item["total"]
                    self.current_order.items.pop(idx)
                    return f"Removed {item['urun']} from your order."
                else:
                    item["quantity"] -= quantity
                    item["total"] = item["quantity"] * item["Fiyat"]
                    self.current_order.total -= (quantity * item["Fiyat"])
                    return f"Removed {quantity}x {item['urun']} from your order."
        return f"Couldn't find {item_name} in your current order."

    def get_order_summary(self) -> str:
        """Get current order summary"""
        if not self.current_order.items:
            return "Your order is empty."
        
        summary = "\nCurrent Order:\n"
        for item in self.current_order.items:
            summary += f"• {item['urun']} x{item['quantity']} = {item['total']:.2f} TL\n"
        summary += f"\nTotal: {self.current_order.total:.2f} TL"
        return summary

    def process_message(self, message: str) -> str:
        """Process customer message and return response"""
        try:
            # Prepare input dictionary with all required keys
            inputs = {
                "question": message,
                "current_order": self.get_order_summary(),
                "chat_history": self.memory.load_memory_variables({})["chat_history"],
                "human_input":""
            }
            
            # Make sure we have all required keys
            response = self.qa_chain(inputs)
            
            # Parse response for order actions
            try:
                actions = json.loads(response["answer"])
                if "add_items" in actions:
                    for item in actions["add_items"]:
                        self.add_to_order(item["urun"], item["quantity"])
                if "remove_items" in actions:
                    for item in actions["remove_items"]:
                        self.remove_from_order(item["urun"], item.get("quantity"))
                return actions.get("response", "Order updated successfully.")
            except json.JSONDecodeError:
                return response["answer"]
            
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def run(self):
        """Run the interactive session"""
        print("Restaurant Assistant: Welcome! How can I help you today?")
        print("\nMenu Categories:")
        categories = set(item.category for item in self.menu_items)
        for category in categories:
            print(f"- {category}")
        
        while True:
            try:
                user_input = input("\nYou: ").strip()
                if user_input.lower() in ["exit", "quit", "bye"]:
                    print("\nThank you for visiting! Goodbye!")
                    break
                    
                response = self.process_message(user_input)
                print(f"\nAssistant: {response}")
                
            except Exception as e:
                print(f"\nAn error occurred: {str(e)}")

# Kullanım örneği
if __name__ == "__main__":
    
    # RestaurantRAG sistemini başlatın
    restaurant = RestaurantRAG("menu1.csv")
    restaurant.run()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


ValueError: could not convert string to float: '120 TL'